In [ ]:
import numpy as np
import pandas as pd

# Load and preprocess data
pivot_df = pd.read_csv('fifty_temperatures_motes.csv')
pivot_df = pivot_df.apply(lambda x: x.fillna(x.mean()), axis=0)

penalty = 0.5
num_nodes = 50
transmission_probs = np.full(num_nodes, 1.0)  # Assume perfect transmission success

# Energy parameters in Joules
E_max = 162000
E_t = 50 / 1000
E_s = 10 / 1000
E_w = 10 / 1000
E_0 = 1 / 1000

# List of M values
M_values = [1, 2, 5, 10]

# Function to simulate Round Robin and calculate average lifetime
def run_simulation_round_robin_lifetime(transmission_probs, num_nodes, M):
    polling_counts = {mote: 0 for mote in range(1, num_nodes + 1)}
    total_packets_rr = 0

    def transmission_success(prob):
        return np.random.rand() < prob

    for t in range(1, len(pivot_df)):
        nodes_rr = [(t - 1 + i) % num_nodes + 1 for i in range(M)]
        for node_rr in nodes_rr:
            if transmission_success(transmission_probs[node_rr - 1]):
                total_packets_rr += 1
            polling_counts[node_rr] += 1

    time_steps = len(pivot_df)
    fw = {mote: polling_counts[mote] / time_steps for mote in polling_counts}
    
    average_lifetime_hours = np.mean([
        E_max / (fw[mote] * (E_t + E_s + E_w) + (1 - fw[mote]) * E_0) for mote in fw
    ]) / 3600
    average_lifetime_years = average_lifetime_hours / 8760

    return average_lifetime_years

# Function to simulate Whittle AoII and calculate average lifetime
def run_simulation_whittle_aoii_lifetime(pivot_df, transmission_probs, num_nodes, M=5, L=100000):
    total_packets_whittle_aoii = 0
    polling_counts = {mote: 0 for mote in range(1, num_nodes + 1)}
    last_update_times = {i: 0 for i in range(1, num_nodes + 1)}
    state_node = {f'mote{i}': np.array([20.0, 0.1]) for i in range(1, num_nodes + 1)}
    state_sink = {f'mote{i}': np.array([20.0, 0.1]) for i in range(1, num_nodes + 1)}

    for t in range(len(pivot_df)):
        whittle_indices = {}
        for mote in range(1, num_nodes + 1):
            column_name = f'mote{mote}'
            delta_t_dynamic = t - last_update_times[mote]
            current_aoii = predict_node_aoii_from_sink(state_sink[column_name], delta_t_dynamic)
            future_aoii_passive = predict_node_aoii_from_sink(state_sink[f'mote{mote}'], delta_t_dynamic + 1)
            future_aoii_active = 0 if np.random.rand() < transmission_probs[mote - 1] else future_aoii_passive
            q_passive = current_aoii + future_aoii_passive
            q_active = current_aoii + future_aoii_active + penalty
            whittle_indices[mote] = q_passive - q_active

        nodes_to_poll = [mote for mote in whittle_indices if whittle_indices[mote] >= 0]
        if len(nodes_to_poll) > M:
            nodes_to_poll = sorted(nodes_to_poll, key=whittle_indices.get, reverse=True)[:M]

        for mote in nodes_to_poll:
            column_name = f'mote{mote}'
            measured_value = pivot_df.loc[t, column_name]
            last_state_value, last_rate_of_change = state_node[column_name]
            delta_t_dynamic = t - last_update_times[mote]

            current_aoii = predict_node_aoii_from_sink(state_sink[column_name], delta_t_dynamic)
            state_node[column_name] = update_node_state_dewma(measured_value, last_state_value, last_rate_of_change, delta_t=1)
            state_sink[column_name] = state_node[column_name]

            polling_counts[mote] += 1
            last_update_times[mote] = t

            if np.random.rand() < transmission_probs[mote - 1]:
                total_packets_whittle_aoii += 1

    time_steps = len(pivot_df)
    fw = {mote: polling_counts[mote] / time_steps for mote in polling_counts}

    average_lifetime_hours = np.mean([
        E_max / (fw[mote] * (E_t + E_s + E_w) + (1 - fw[mote]) * E_0) for mote in fw
    ]) / 3600
    average_lifetime_years = average_lifetime_hours / 8760

    return average_lifetime_years

# Run simulations for each M value
for M in M_values:
    print(f"\nRunning simulation for M = {M}")
    
    # Run Round Robin lifetime calculation
    average_lifetime_rr = run_simulation_round_robin_lifetime(transmission_probs, num_nodes, M)
    print(f"Average Sensor Lifetime (years) for Round Robin (M={M}): {average_lifetime_rr:.2f}")
    
    # Run Whittle AoII lifetime calculation
    average_lifetime_whittle_aoii = run_simulation_whittle_aoii_lifetime(pivot_df, transmission_probs, num_nodes, M)
    print(f"Average Sensor Lifetime (years) for Whittle AoII (M={M}): {average_lifetime_whittle_aoii:.2f}")


In [ ]:
import numpy as np
import pandas as pd

# Load and preprocess data
pivot_df = pd.read_csv('fifty_temperatures_motes.csv')
pivot_df = pivot_df.apply(lambda x: x.fillna(x.mean()), axis=0)

penalty = 0.5
num_nodes = 50
transmission_probs = np.full(num_nodes, 1.0)  # Assume perfect transmission success

# Energy parameters in Joules
E_max = 162000
E_t = 50 / 1000
E_s = 10 / 1000
E_w = 10 / 1000
E_0 = 1 / 1000

# List of M values
M_values = [1, 2, 5, 10]

# Function to simulate Round Robin and calculate average lifetime
def run_simulation_round_robin_lifetime(transmission_probs, num_nodes, M):
    polling_counts = {mote: 0 for mote in range(1, num_nodes + 1)}
    total_packets_rr = 0

    def transmission_success(prob):
        return np.random.rand() < prob

    for t in range(1, len(pivot_df)):
        nodes_rr = [(t - 1 + i) % num_nodes + 1 for i in range(M)]
        for node_rr in nodes_rr:
            if transmission_success(transmission_probs[node_rr - 1]):
                total_packets_rr += 1
            polling_counts[node_rr] += 1

    time_steps = len(pivot_df)
    fw = {mote: polling_counts[mote] / time_steps for mote in polling_counts}
    
    average_lifetime_hours = np.mean([
        E_max / (fw[mote] * (E_t + E_s + E_w) + (1 - fw[mote]) * E_0) for mote in fw
    ]) / 3600
    average_lifetime_years = average_lifetime_hours / 8760

    return average_lifetime_years

# Function to simulate Whittle AoII and calculate average lifetime
def run_simulation_whittle_aoii_lifetime(pivot_df, transmission_probs, num_nodes, M=5, L=100000):
    total_packets_whittle_aoii = 0
    polling_counts = {mote: 0 for mote in range(1, num_nodes + 1)}
    last_update_times = {i: 0 for i in range(1, num_nodes + 1)}
    state_node = {f'mote{i}': np.array([20.0, 0.1]) for i in range(1, num_nodes + 1)}
    state_sink = {f'mote{i}': np.array([20.0, 0.1]) for i in range(1, num_nodes + 1)}

    for t in range(len(pivot_df)):
        whittle_indices = {}
        for mote in range(1, num_nodes + 1):
            column_name = f'mote{mote}'
            delta_t_dynamic = t - last_update_times[mote]
            current_aoii = predict_node_aoii_from_sink(state_sink[column_name], delta_t_dynamic)
            future_aoii_passive = predict_node_aoii_from_sink(state_sink[f'mote{mote}'], delta_t_dynamic + 1)
            future_aoii_active = 0 if np.random.rand() < transmission_probs[mote - 1] else future_aoii_passive
            q_passive = current_aoii + future_aoii_passive
            q_active = current_aoii + future_aoii_active + penalty
            whittle_indices[mote] = q_passive - q_active

        nodes_to_poll = [mote for mote in whittle_indices if whittle_indices[mote] >= 0]
        if len(nodes_to_poll) > M:
            nodes_to_poll = sorted(nodes_to_poll, key=whittle_indices.get, reverse=True)[:M]

        for mote in nodes_to_poll:
            column_name = f'mote{mote}'
            measured_value = pivot_df.loc[t, column_name]
            last_state_value, last_rate_of_change = state_node[column_name]
            delta_t_dynamic = t - last_update_times[mote]

            current_aoii = predict_node_aoii_from_sink(state_sink[column_name], delta_t_dynamic)
            state_node[column_name] = update_node_state_dewma(measured_value, last_state_value, last_rate_of_change, delta_t=1)
            state_sink[column_name] = state_node[column_name]

            polling_counts[mote] += 1
            last_update_times[mote] = t

            if np.random.rand() < transmission_probs[mote - 1]:
                total_packets_whittle_aoii += 1

    time_steps = len(pivot_df)
    fw = {mote: polling_counts[mote] / time_steps for mote in polling_counts}

    average_lifetime_hours = np.mean([
        E_max / (fw[mote] * (E_t + E_s + E_w) + (1 - fw[mote]) * E_0) for mote in fw
    ]) / 3600
    average_lifetime_years = average_lifetime_hours / 8760

    return average_lifetime_years

# Run simulations for each M value
for M in M_values:
    print(f"\nRunning simulation for M = {M}")
    
    # Run Round Robin lifetime calculation
    average_lifetime_rr = run_simulation_round_robin_lifetime(transmission_probs, num_nodes, M)
    print(f"Average Sensor Lifetime (years) for Round Robin (M={M}): {average_lifetime_rr:.2f}")
    
    # Run Whittle AoII lifetime calculation
    average_lifetime_whittle_aoii = run_simulation_whittle_aoii_lifetime(pivot_df, transmission_probs, num_nodes, M)
    print(f"Average Sensor Lifetime (years) for Whittle AoII (M={M}): {average_lifetime_whittle_aoii:.2f}")
